<a href="https://colab.research.google.com/github/anamkhan315/test2/blob/main/tracking_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import folium
from IPython.display import display, HTML

# Step 1: Load the data
data = pd.read_csv('/content/drive/MyDrive/Whales CSV.csv')  # Replace with your CSV file
data['time'] = pd.to_datetime(data['time'])
data = data.sort_values(by='time')

# Step 2: Normalize the latitude and longitude
scaler = MinMaxScaler()
data[['latitude', 'longitude']] = scaler.fit_transform(data[['latitude', 'longitude']])

# Step 3: Create sequences for LSTM
def create_sequences(data, time_steps):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        seq = data[i:i + time_steps]
        label = data[i + time_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

time_steps = 10  # Number of past observations to consider
features = data[['latitude', 'longitude']].values
sequences, labels = create_sequences(features, time_steps)

# Step 4: Split data into training and test sets
split = int(len(sequences) * 0.8)
X_train, X_test = sequences[:split], sequences[split:]
y_train, y_test = labels[:split], labels[split:]

# Step 5: Build the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(time_steps, 2), return_sequences=True),
    LSTM(50, activation='relu'),
    Dense(2)  # Output layer for latitude and longitude
])

model.compile(optimizer='adam', loss='mse')

# Step 6: Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Step 7: Predict future paths
future_steps = 20  # Number of future steps to predict
input_seq = X_test[-1]  # Start with the last sequence in the test set
predicted_future = []

for _ in range(future_steps):
    prediction = model.predict(input_seq[np.newaxis, :, :])
    predicted_future.append(prediction[0])
    input_seq = np.vstack((input_seq[1:], prediction))  # Append prediction to sequence

predicted_future = np.array(predicted_future)

# Step 8: Inverse transform predictions
predicted_coordinates = scaler.inverse_transform(predicted_future)
true_coordinates = scaler.inverse_transform(y_test)
last_known = scaler.inverse_transform(X_test[-1])

# Step 9: Plot results on a real-time map
# Create a map centered on the last known position
last_lat, last_lon = last_known[-1]
whale_map = folium.Map(location=[last_lat, last_lon], zoom_start=6)

# Add actual test points to the map
for lat, lon in true_coordinates[:future_steps]:
    folium.CircleMarker(location=[lat, lon], radius=3, color='blue', fill=True, fill_opacity=0.6).add_to(whale_map)


# Add predicted future points to the map
for lat, lon in predicted_coordinates:
    folium.CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_opacity=0.6).add_to(whale_map)

# Save and display the map
whale_map.save('whale_tracking_with_predictions.html')
whale_map

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.2152 - val_loss: 0.0964
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0647 - val_loss: 0.0545
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0224 - val_loss: 0.0124
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0134 - val_loss: 0.0122
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0117 - val_loss: 0.0151
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0133 - val_loss: 0.0113
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0123 - val_loss: 0.0113
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0128 - val_loss: 0.0111
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0115 - val_loss: 0.0106
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0114 - val_loss: 0.0102
Epoch 11/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0103 - val_loss: 0.0108
Epoch 12/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0104 - val_l